In [ ]:
import asyncio
import logging
import sys

from astropy.coordinates import Angle
from astropy.time import Time
import astropy.units as u

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS
from lsst.ts.observatory.control.maintel.comcam import ComCam

import mt_utils

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()
#mtcs = MTCS(domain)
comcam = ComCam(domain)
mtptg = salobj.Remote(domain, "MTPtg")
newmtmount = salobj.Remote(domain, "NewMTMount")

#await asyncio.gather(mtcs.start_task, comcam.start_task)
await asyncio.gather(comcam.start_task, mtptg.start_task, newmtmount.start_task)

In [ ]:
date = Time.now()
group_id = f'CALSET_{date.tai.strftime("%Y%m%d_%H%M")}'

In [ ]:
await comcam.take_bias(nbias=1, group_id=group_id)

In [ ]:
await comcam.take_darks(exptime=5.0, ndarks=2, group_id=group_id)

In [ ]:
ra_dec = mt_utils.altaz_to_radec(39.0, 205.7)

mtptg.cmd_raDecTarget.set(
    ra=ra_dec.ra.hour,
    declination=ra_dec.dec.deg,
    rotPA=Angle(-180.0, unit=u.deg).deg,
    targetName="Flatfield position",
    frame=2,
    epoch=2000,
    equinox=2000,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotFrame=1,
    rotMode=1,
)

# mtptg.cmd_azElTarget.set(
#     targetName="Flatfield position",
#     azDegs=Angle(205.7, unit=u.deg).deg,
#     elDegs=Angle(39.0, unit=u.deg).deg,
#     rotPA=Angle(0.0, unit=u.deg).deg
# )

In [ ]:
await mt_utils.slew(mtptg, newmtmount, True)
#ack = await mtptg.cmd_azElTarget.start(timeout=30)
#print(ack)

In [ ]:
await comcam.take_flats(exptime=2.0, nflats=2, group_id=group_id)

In [ ]:
#await comcam.take_spot(exptime=3.0, n=2, group_id=group_id)

In [ ]:
#await comcam.take_engtest(exptime=2.0, n=1, group_id=group_id, test_type="FE55", obs_note="Fake test")